# Toronto Neigborhoods Project

<b>I have used Beautiful Soup Package</b>

In [1]:
#install Beautiful Soup Package and parser
!pip install bs4 lxml

In [2]:
#import libraries
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [3]:
#read webpage into a variable soup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = get(url)

#read the content into soup
soup = BeautifulSoup(response.text, 'html.parser')

#find the first table
table = soup.find('table')

#get all rows
table_rows = table.find_all('tr')  

#read rows to a dataframe
df = pd.read_html(str(table))[0]    

#drop boroughs not assigned
df2 = df[df.Borough != 'Not assigned'].reset_index(drop=True)

#check the final dataframe
df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df2.shape

(103, 3)

In [5]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pgeocode
nomi = pgeocode.Nominatim('ca')
nomi.query_postal_code("M4A")

postal_code                                            M4A
country code                                            CA
place_name        North York (Sweeney Park / Wigmore Park)
state_name                                         Ontario
state_code                                              ON
county_name                                            NaN
county_code                                            NaN
community_name                                         NaN
community_code                                         NaN
latitude                                           43.7276
longitude                                         -79.3148
accuracy                                                 6
Name: 0, dtype: object

In [7]:
for index, row in df2.iterrows():
    lat = nomi.query_postal_code(row['Postal Code'])['latitude']
    lon = nomi.query_postal_code(row['Postal Code'])['longitude']
    #print(lat,lon)
    df2.at[index,'Latitude'] = lat
    df2.at[index,'Longitude'] = lon
df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


In [8]:
#filter Boroughs that contain Toronto only
df3 = df2[df2['Borough'].str.contains("Toronto")]
df3

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
15,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
19,M4E,East Toronto,The Beaches,43.6784,-79.2941
20,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754
24,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860
25,M6G,Downtown Toronto,Christie,43.6683,-79.4205
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6496,-79.3833
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6655,-79.4378


In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Lib

In [13]:
# lat and lon for toronto
t_lat = 43.6532
t_lon = -79.3832

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[t_lat, t_lon], zoom_start=11)

#drop the wrong data entry
df3.drop(df3.index[-1],inplace=True)

# add markers to map
for lat, lng, label in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
CLIENT_ID = 'VBXSTWM51FOOBV2MZLUMQT5UITMYSXIILAVW04KORPQLP42S' # your Foursquare ID
CLIENT_SECRET = 'C4LB20JG5AJVL3SNEYVOJ4PN5ANB5QIM3D1QOTY15AWAXELA' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: VBXSTWM51FOOBV2MZLUMQT5UITMYSXIILAVW04KORPQLP42S
CLIENT_SECRET:C4LB20JG5AJVL3SNEYVOJ4PN5ANB5QIM3D1QOTY15AWAXELA


In [22]:
df3 = df3.reset_index(drop=True)

In [23]:
neighborhood_latitude = df3.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df3.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df3.loc[0, 'Neighborhood'] # neighborhood name
zip_code = df3.loc[0, 'Postal Code 	'] # postal code

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6555, -79.3626.


In [27]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
query = "school" #seacrh for schools in first ZIP
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&query={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    query,
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?&client_id=VBXSTWM51FOOBV2MZLUMQT5UITMYSXIILAVW04KORPQLP42S&client_secret=C4LB20JG5AJVL3SNEYVOJ4PN5ANB5QIM3D1QOTY15AWAXELA&v=20180604&query=school&ll=43.6555,-79.3626&radius=500&limit=100'

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eba84a914a126001bb43ef9'},
 'response': {'venues': [{'id': '4fe873dce4b0ec3d85f23caa',
    'name': 'George Brown College - School of ESL',
    'location': {'address': '341 King St. E',
     'lat': 43.65187195086978,
     'lng': -79.36557973642425,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65187195086978,
       'lng': -79.36557973642425}],
     'distance': 469,
     'postalCode': 'M5A 1L1',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['341 King St. E', 'Toronto ON M5A 1L1', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d198941735',
      'name': 'College Academic Building',
      'pluralName': 'College Academic Buildings',
      'shortName': 'Academic Building',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/academicbuilding_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1589282040',
    'hasPerk': F

In [42]:
schools = results['response']['venues']
nearby_venues = pd.json_normalize(schools) # flatten JSON
nearby_venues.head(1)



,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4fe873dce4b0ec3d85f23caa,George Brown College - School of ESL,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C...",v-1589282040,False,341 King St. E,43.651872,-79.36558,"[{'label': 'display', 'lat': 43.65187195086978...",469,M5A 1L1,CA,Toronto,ON,Canada,"[341 King St. E, Toronto ON M5A 1L1, Canada]",NaN


In [44]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in nearby_venues.columns if col.startswith('location.')] + ['id']
dataframe_filtered = nearby_venues.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,George Brown College - School of ESL,College Academic Building,341 King St. E,43.651872,-79.365580,"[{'label': 'display', 'lat': 43.65187195086978...",469,M5A 1L1,CA,Toronto,ON,Canada,"[341 King St. E, Toronto ON M5A 1L1, Canada]",NaN,4fe873dce4b0ec3d85f23caa
1,St Paul Catholic School,Elementary School,NaN,43.656199,-79.361970,"[{'label': 'display', 'lat': 43.65619913310336...",92,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4f9fd7bee4b0a37923e166b6
2,Regent Park / Duke of York Junior Public School,Middle School,20 Regent St.,43.657764,-79.363933,"[{'label': 'display', 'lat': 43.65776407715354...",273,M5A 3N6,CA,Toronto,ON,Canada,"[20 Regent St. (at Shuter), Toronto ON M5A 3N6...",at Shuter,4d24ed4ec0772c0f2cba9c88
3,Inglenook Community School,High School,19 Sackville,43.653903,-79.359429,"[{'label': 'display', 'lat': 43.65390281374826...",311,NaN,CA,Toronto,ON,Canada,"[19 Sackville, Toronto ON, Canada]",NaN,4bd6ebaf6f6495213a8c71ec
4,Market Lane Public School,Elementary School,246 The Esplanade,43.653344,-79.364758,"[{'label': 'display', 'lat': 43.65334399982201...",296,NaN,CA,Toronto,ON,Canada,"[246 The Esplanade, Toronto ON, Canada]",NaN,4af59f14f964a52070fa21e3
5,George Brown School of Design,College Technology Building,333 King St. East,43.651871,-79.365797,"[{'label': 'display', 'lat': 43.65187113930064...",479,NaN,CA,Toronto,ON,Canada,"[333 King St. East (Ontario St.), Toronto ON, ...",Ontario St.,4e67932122715179703b175f
6,Nelson Mandela Public School,Elementary School,NaN,43.658111,-79.360072,"[{'label': 'display', 'lat': 43.65811084259157...",354,NaN,CA,NaN,NaN,Canada,[Canada],NaN,524060b1498e092fb0e8cd80
7,Regent Park School of Music,Music Venue,534 Queen Street East,43.656912,-79.357168,"[{'label': 'display', 'lat': 43.65691161146316...",464,NaN,CA,Toronto,ON,Canada,"[534 Queen Street East, Toronto ON, Canada]",NaN,4eef45335c5c794ad4305a1d
8,Lord Dufferin Public School,School,Parliament Ave,43.659383,-79.367464,"[{'label': 'display', 'lat': 43.65938343053729...",583,NaN,CA,Toronto,ON,Canada,"[Parliament Ave, Toronto ON, Canada]",NaN,4f3ac383e4b0eebd6cd4242c
9,George Brown School Of Design,College Technology Building,341 King St. E,43.651895,-79.365601,"[{'label': 'display', 'lat': 43.65189490212039...",468,M5A 1L1,CA,Toronto,ON,Canada,"[341 King St. E, Toronto ON M5A 1L1, Canada]",NaN,4fd91494e4b0392d37a814de


In [55]:
#get all elementaru middle and high schools
es = dataframe_filtered.loc[dataframe_filtered['categories'] == 'Elementary School']
ms = dataframe_filtered.loc[dataframe_filtered['categories'] == 'Middle School']
hs = dataframe_filtered.loc[dataframe_filtered['categories'] == 'High School']

print(es.shape, ms.shape, hs.shape)

(3, 15) (1, 15) (1, 15)


In [57]:
schools_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add elementary schools as blue circle markers
for lat, lng, label in zip(es.lat, es.lng, es.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(schools_map)

# add middle schools as red circle markers
for lat, lng, label in zip(ms.lat, ms.lng, ms.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(schools_map)

# add high schools as green circle markers
for lat, lng, label in zip(hs.lat, hs.lng, hs.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(schools_map)
    
# display map
schools_map